In [2]:
import cv2
from PIL import Image
import numpy as np
import uuid
from keras.models import load_model
from mtcnn import MTCNN
from sklearn.manifold import TSNE

import skimage
import sklearn
import pandas as pd  
import tensorflow as tf

import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
from embedding import l2_normalize,face_embedding
from save_file import save_data

# Process the face detection on a video
1. Load the numpy array generated from a video
1. Process the face detection
1. Visual check of the performances


## Load the numpy array : 3D + 3D numpy with colors or 3D if gray

In [4]:
USE_COLOR = True 
SHOW_DET_FACES = True
fps_np = 1.

# Load the numpy array 
path_arrays = "../data/arrays/"
array_name = 'TH_GNS_' + str(int(fps_np)) + 'fps'
#array_name = 'EJ_GNS_' + str(int(fps_np)) + 'fps'

if USE_COLOR:
    color = '_RGB'
else:
    color = '_Gray'
    
# Load the numpy video 
file_in_np = path_arrays + array_name + color + '.npy'
video_np =np.load(file = file_in_np)
video_np = video_np.astype(np.uint8)


## Load the DNN model


In [5]:
# deep face 
#model_path = '../data/models/deepface/keras/deepface_fn_512.h5'
#model_path = '../data/models/deepface/keras/deepface_fn_128.h5'

# face net
#model_path = '../data/models/facenet/keras/facenet_ds_keras_128.h5'
model_path = '../data/models/facenet/keras/facenet_ds_keras_512.h5'

model_emb = load_model(model_path, compile = False , custom_objects={"tf": tf})

/Users/aro/.local/share/virtualenvs/benchmark-face-reco-f4UNL1EF/lib/python3.9/site-packages/keras/layers/core/lambda_layer.py:297: UserWarning: FaceNet is not loaded, but a Lambda layer uses it. It may cause errors.
  function = cls._parse_function_from_config(config, custom_objects,
2022-02-09 23:48:39.685760: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Metadata

In [16]:
# Save files : embeddings and metadata
file_emb_np = path_arrays + array_name + '_emb.npy'
file_time = path_arrays + array_name + '_time.npy'
file_uuid = path_arrays + array_name + '_uuid.npy'
file_out_json = path_arrays + array_name + 'emb.json'



# Contour margin for the face cropping
margin = 0
# mininum size of the face to be
min_face_size = 20
image_size = 160 # embeddings


metadata_video = {
                  'video_uri': None,
                  'filename_metadata': file_out_json,
                  'filename_embeddings': file_emb_np,
                  'face_detection' : 'MTCNN',
                  'model_emb': 'facenet_ds_keras_512.h5',
                  'use_color': USE_COLOR,
                  'crop_margin': margin,
                  'min_face_size': min_face_size,
                  'image_size' : image_size,
                  'fps': fps_np}


In [9]:
# check the size of the video 
if USE_COLOR:
    nb_frame, height, width, color = video_np.shape
else:
    nb_frame, height, width = video_np.shape
    color = 1

print('Number of frame : ' + str(nb_frame))
print('Frame size WxH : ' + str(width) + 'x' + str(height))
print('Colors : ' + str(color))


Number of frame : 153
Frame size WxH : 320x180
Colors : 3


## Process the video

In [10]:
# models

face_detector = MTCNN(min_face_size=min_face_size)

model_emb = metadata_video['model_emb']
model_path = '../data/models/facenet/keras/' + model_emb
model_emb = load_model(model_path, compile=False, custom_objects={"tf": tf})

In [11]:
# outputs
dic_crop_faces = {}
video_fd = np.copy(video_np)
uuid_faces = []
faces_emb = []
time = []
for iname, image_np in enumerate(video_np):

    # for videos : image_np = frame_np
    face_crop, face_emb = face_embedding(image_np, face_detector,
                                         model_emb,
                                         max_nb_faces=10000,
                                         margin=margin,
                                         image_size=image_size,
                                         flag_normalise=False,
                                         flag_plot=False)

    if len(face_emb) > 0:
        uuid_faces.append([uuid.uuid4() for k in range(len(face_emb))])
        faces_emb.append(face_emb)

        for k in range(np.shape(face_emb)[0]):
            time.append(iname)

In [18]:
faces_emb_np = np.concatenate(faces_emb, axis=0)

In [19]:
save_data(file_emb_np, faces_emb_np, file_out_json, metadata_video)
np.save(file=file_time, arr=time)
np.save(file=file_uuid, arr=uuid_faces)